# Import packages and data sets

In [ ]:
!pip install lightgbm
!pip install shap

In [ ]:
import glob
import pandas as pd
import numpy as np
import lightgbm as lgbm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import os
import sys
import shap
import joblib


raw_dir = "/home/ec2-user/pwp-summer-2019/master_thesis_nhh_2019/raw_data/" 
data_dir = "/home/ec2-user/pwp-summer-2019/master_thesis_nhh_2019/processed_data/" 

pd.set_option('display.max_columns', 999)

In [ ]:
# Import the class feature_engineering from the file "Functions"
from Functions import (feature_engineering)

In [ ]:
df_train = pd.read_pickle(data_dir+'df_train')
df_val = pd.read_pickle(data_dir+'df_val')
df_test = pd.read_pickle(data_dir+'df_test')

formation_dictionary = joblib.load(data_dir+'formation_dictionary.pkl')

df_train_val = df_train.append(df_val)

# Feature engineering and remove outliers

In [ ]:
params_features = {
    'outlier_values': {'gr': df_train_val.gr.quantile(0.9995),
                       'rmed': df_train_val.rmed.quantile(0.9995),
                        'rdep': df_train_val.rdep.quantile(0.9995)
                      },
    'above_below_variables': ['gr','rdep','rmed'],
    'y_variable': 'formation_2',
    'num_shifts': 1,
    'cols_to_remove' : ['depth', 'dts','hgr', 'hnphi', 
                        'hrdep', 'hrhob', 'hrmed', 'hrsh','rsh','field','main_area','md'],
    'thresh': 7,
    'var1_ratio': 'gr'
}

### For home-made stratified split

In [ ]:
train_class = feature_engineering(df_train,**params_features)

train_class.remove_outliers()
train_class.above_below()
train_class.cleaning()
train_class.xyz()

df_train = train_class.df
columns_class = df_train.columns

val_class = feature_engineering(df_val,**params_features)

val_class.remove_outliers()
val_class.above_below()
val_class.cleaning()
val_class.xyz()
df_val = val_class.df[columns_class]

### For sklearn(randomized) stratified split

In [ ]:
df_class = feature_engineering(df_train_val,**params_features)

df_class.remove_outliers()
df_class.above_below()
df_class.cleaning()
df_class.xyz()

df = df_class.df

# Split into train_valid/test

### For home-made stratified split

In [ ]:
col = ['formation','title','formation_2','group']
X_train = df_train.drop(col, axis=1)
Y_train = df_train['formation_2']

X_valid = df_val.drop(col, axis=1)
Y_valid = df_val['formation_2']

features_list = X_train.columns

### For sklearn(randomized) stratified split

In [ ]:
col = ['formation','title','formation_2','group']
X = df.drop(col, axis=1)
y = df['formation_2']

In [ ]:
X_train_and_valid, X_test, Y_train_and_valid, Y_test = train_test_split( X, y, 
                                                                        test_size=0.10, 
                                                                        random_state=42, 
                                                                        stratify=y)

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_and_valid, Y_train_and_valid, 
                                                      test_size=0.33, 
                                                      random_state=42, 
                                                      stratify=Y_train_and_valid)

# Compute class weights

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(Y_train),Y_train)
class_weights_dict=dict(zip(np.unique(Y_train), class_weights))

# Train model

In [ ]:
eval_set = [(X_train, Y_train), (X_valid, Y_valid)]

# Tuning from Bayesian
model_lightgbm = lgbm.LGBMClassifier(boosting_type='gbdt',
                            learning_rate = 0.05,
                            num_leaves = 5,
                            n_estimators = 500, 
                            boost_from_average = False,
                            silent = False,
                            max_depth =  90, #12
                            class_weight=class_weights_dict,
                            feature_fraction= 1,
                            min_data_in_leaf = 657,
                            random_state = 42
                                             )

model_lightgbm.fit(X = X_train, y = Y_train,
         eval_set = eval_set,
         early_stopping_rounds= 50,
         eval_metric= 'multi_logloss',
         verbose = 10
         )

In [ ]:
# Save model
joblib.dump(model_lightgbm, 'lgbm_model.pkl')
# Load model
#model_lightgbm = joblib.load('lgbm_model.pkl')

# Visualize training and validation loss 

In [ ]:
pd.Series(model_lightgbm.evals_result_['training']['multi_logloss']).plot()
pd.Series(model_lightgbm.evals_result_['valid_1']['multi_logloss']).plot()

# Test data

### Function to compute test results 

In [ ]:
from sklearn import metrics
def predict_and_post_process(data_,real_values,formation_code,model_, 
                             features_list_, result, test_data = 'no'):
    data = data_.copy()
    
    if result == 'light_gbm':
    
        train_predict = model_.predict(data)
        train_predict_df = pd.DataFrame(train_predict)
        #train_predict_df['target_lgbm'] = train_predict_df.values
        #train_predict_df = pd.DataFrame(train_predict)
        train_predict_df['target_lgbm'] = train_predict_df.idxmax(axis=1)
    
    elif result == 'grid_result':
        train_predict = model_.predict(data)
        train_predict_df = pd.DataFrame(train_predict)
        #train_predict_df['target_lgbm'] = train_predict_df.values
        #train_predict_df = pd.DataFrame(train_predict)
        train_predict_df['target_lgbm'] = train_predict_df
        
    if test_data == 'yes':
        f1 = metrics.f1_score(train_predict_df['target_lgbm'], real_values, average='micro')
        print ('f1 score is: ', f1)
    

    test_df = pd.DataFrame(data)
    test_df['target_lgbm'] = train_predict_df['target_lgbm'].values
    test_df['target'] = real_values.values
    
    print ('total points:', len(test_df))
    test_df['predicted_formation'] = test_df['target_lgbm'].map(formation_code)
    
    if type(list(real_values)[0]) == int:
        f1 = metrics.f1_score(train_predict_df['target_lgbm'], real_values, average='micro')
        print ('f1 score is: ', f1)
        test_df['real_formation'] = test_df['target'].map(formation_code)
        print ('Number of points not matching: ', 
               len(test_df.loc[~(test_df['real_formation'] == test_df['predicted_formation'])]))

    else:
        test_df['real_formation'] = test_df['target'].values
        
        f1 = metrics.f1_score(test_df['predicted_formation'], real_values, average='micro')
        print ('f1 score is: ', f1)
        print ('Number of points not matching: ', 
               len(test_df.loc[~(test_df['real_formation'] == test_df['predicted_formation'])]))
        print ('---------------------------------')
        
    if test_data == 'no':
        print ('Point in real formation: \n', test_df['real_formation'].value_counts())
        print ('Point in predicted formation: \n', test_df['predicted_formation'].value_counts())
    
    #new_cols=['ac', 'den', 'gr', 'neu', 'rdep', 'rmed', 'tvd','x','y','z']
    test_df.rename(columns=dict(zip(test_df.columns[:len(features_list_)], 
                                    features_list_)),inplace=True)
    
    return test_df, f1

## Blind data

In [ ]:
blind_wells = df_test.title.unique()

test_class = feature_engineering(df_test,**params_features)

test_class.above_below()
test_class.thresh = 0 # In order to not remove any rows when cleaning
test_class.cleaning()
test_class.xyz()

### Predict on full test set

In [ ]:
exp_well = test_class.df.copy()

XX = exp_well.drop(col,axis = 1).values
yy = exp_well['formation_2']

print ('-----------')
blind_data_prediction, f1 = predict_and_post_process(XX,yy,formation_dictionary,model_lightgbm,
                                                     features_list, 'grid_result') ## for grid result model

In [ ]:
# Save predictions to data set - Not stratified
test_df = test_class.df
test_df['predicted'] = blind_data_prediction.target_lgbm

#test_df.to_pickle('blind_test_lgbm_data')

### Test on individual wells:

In [ ]:
blind_wells = df_test.title.unique()
test_class = feature_engineering(df_test,**params_features)

test_class.above_below()
test_class.thresh = 0 # In order to not remove any rows when cleaning
test_class.cleaning()
test_class.xyz()

test_df = test_class.df

results = {}

for well in blind_wells:

    exp_well = test_class.df[test_class.df.title == well].copy() 

    XX = exp_well.drop(col,axis = 1).values
    yy = exp_well['formation_2']

    print ('-----------')
    blind_data_prediction, f1 = predict_and_post_process(XX,yy,formation_dictionary,model_lightgbm,
                                      features_list, 'grid_result') ## for grid result model
    results[well] = f1
    blind_data_prediction

print ('\n-----------')
f1_values = []
for key, values in results.items():
    f1_values.append(values)
print('Mean of all wells:',np.mean(f1_values))
print ('-----------')
results

# SHAP

In [ ]:
# load JS visualization code to notebook
shap.initjs()

# explain the model's predictions using SHAP values
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model_lightgbm)
shap_values = explainer.shap_values(XX)

## Cumulative Feature Importance for "well x"

In [ ]:
import matplotlib.pyplot as plt

# TO CHOOSE COLORPALATE: https://matplotlib.org/examples/color/colormaps_reference.html
#shap.summary_plot(shap_values, X, plot_type="dot",color=pl.get_cmap("tab10"))

shap.summary_plot(shap_values, XX,
                  feature_names = features_list, 
                  plot_type   = "bar", 
                  max_display = None,
                  plot_size   = (12,10),#'auto',
                  class_names = list(formation_dictionary.values()),
                  color       = plt.get_cmap("tab20")
                  )

## Feature importance for specific class

What feature levels drove our predicitons twoards predicting the given class

In [ ]:
shap.summary_plot(shap_values[29], XX, feature_names=features_list)